# Fake News Analysis and Classification

This notebook performs exploratory data analysis (EDA) on the fake news dataset and evaluates different classification models.

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the parent directory to the path to import the models module
sys.path.append('..')
from models import load_data, LogisticModel, DistilBertModel

## 1. Load and Explore the Dataset

In [ ]:
# Load the dataset from TSV files
data = load_data('../data', use_predefined_splits=True)
df = data['full_data']

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nOriginal label distribution:\n{df['label'].value_counts()}")
print(f"\nOriginal label distribution (percentage):\n{df['label'].value_counts(normalize=True) * 100}")

print(f"\nBinary label distribution:\n{df['binary_label'].value_counts()}")
print(f"\nBinary label distribution (percentage):\n{df['binary_label'].value_counts(normalize=True) * 100}")

# Display split sizes
print(f"\nTrain set size: {len(data['train']['df'])}")
print(f"Test set size: {len(data['test']['df'])}")
print(f"Validation set size: {len(data['valid']['df'])}")

In [ ]:
# Display sample data
df.head()

## 2. Exploratory Data Analysis

In [ ]:
# Text length distribution
df['statement_length'] = df['statement'].apply(len)

plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='statement_length', hue='binary_label', bins=50, kde=True)
plt.title('Statement Length Distribution by Class')
plt.xlabel('Statement Length')
plt.ylabel('Count')
plt.legend(['Real News (0)', 'Fake News (1)'])
plt.show()

In [ ]:
# 1. Check for missing or null values
print("Checking for missing values:")
missing_values = df.isnull().sum()
print(missing_values)
    
# 2. Check label distribution
print("\nOriginal label distribution:")
print(df['label'].value_counts())
print("\nUnique labels:", df['label'].unique())

print("\nBinary label distribution:")
print(df['binary_label'].value_counts())

# 3. Check for duplicates
duplicate_count = df.duplicated(['statement', 'binary_label']).sum()
print(f"\nNumber of duplicate entries (same statement and label): {duplicate_count}")

# Clean the data
# Remove missing values if any
df = df.dropna(subset=['statement', 'binary_label'])

# Remove duplicates if any
df = df.drop_duplicates(subset=['statement', 'binary_label'])

# Show cleaned data stats
print("\nAfter cleaning:")
print(f"Total samples: {len(df)}")
print("\nBinary label distribution after cleaning:")
print(df['binary_label'].value_counts())

# Update the data dictionary with cleaned data
data['full_data'] = df


## 3. Train and Evaluate Logistic Regression Model

In [ ]:
# Initialize and train the Logistic Regression model
logistic_model = LogisticModel(max_features=10000, preprocess=True)
logistic_model.train(data['train']['texts'], data['train']['labels'])

# Evaluate the model on test set
logistic_metrics = logistic_model.evaluate(data['test']['texts'], data['test']['labels'])

# Display metrics
print("Logistic Regression Model Metrics on Test Set:")
for metric, value in logistic_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric.capitalize()}: {value:.4f}")

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    logistic_metrics['confusion_matrix'],
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Real', 'Fake'],
    yticklabels=['Real', 'Fake']
)
plt.title('Confusion Matrix - Logistic Regression (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Evaluate on validation set
logistic_val_metrics = logistic_model.evaluate(data['valid']['texts'], data['valid']['labels'])

# Display validation metrics
print("\nLogistic Regression Model Metrics on Validation Set:")
for metric, value in logistic_val_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric.capitalize()}: {value:.4f}")

# Save the model
logistic_model.save('../models/logistic_model.pkl')

## 4. Train and Evaluate DistilBERT Model

In [ ]:
# Initialize and train the DistilBERT model
# Note: This may take some time to run
distilbert_model = DistilBertModel(max_length=128, batch_size=16, epochs=2)
distilbert_model.train(data['train']['texts'], data['train']['labels'])

# Evaluate the model on test set
distilbert_metrics = distilbert_model.evaluate(data['test']['texts'], data['test']['labels'])

# Display metrics
print("DistilBERT Model Metrics on Test Set:")
for metric, value in distilbert_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric.capitalize()}: {value:.4f}")

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    distilbert_metrics['confusion_matrix'],
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Real', 'Fake'],
    yticklabels=['Real', 'Fake']
)
plt.title('Confusion Matrix - DistilBERT (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Evaluate on validation set
distilbert_val_metrics = distilbert_model.evaluate(data['valid']['texts'], data['valid']['labels'])

# Display validation metrics
print("\nDistilBERT Model Metrics on Validation Set:")
for metric, value in distilbert_val_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric.capitalize()}: {value:.4f}")

# Save the model
distilbert_model.save('../models/distilbert_model')

## 5. Compare Model Performance

In [ ]:
# Compare model performance on test set
metrics = ['accuracy', 'precision', 'recall', 'f1_score']
models = ['Logistic Regression', 'DistilBERT']
test_performance = {
    'Logistic Regression': [logistic_metrics[metric] for metric in metrics],
    'DistilBERT': [distilbert_metrics[metric] for metric in metrics]
}

# Create a DataFrame for test set comparison
test_performance_df = pd.DataFrame(test_performance, index=metrics)
test_performance_df.index = [metric.capitalize() for metric in metrics]

# Display the test set comparison
print("Model Performance Comparison on Test Set:")
test_performance_df


In [ ]:
# Compare model performance on validation set
val_performance = {
    'Logistic Regression': [logistic_val_metrics[metric] for metric in metrics],
    'DistilBERT': [distilbert_val_metrics[metric] for metric in metrics]
}

# Create a DataFrame for validation set comparison
val_performance_df = pd.DataFrame(val_performance, index=metrics)
val_performance_df.index = [metric.capitalize() for metric in metrics]

# Display the validation set comparison
print("Model Performance Comparison on Validation Set:")
val_performance_df


In [ ]:
# Plot the test set comparison
plt.figure(figsize=(12, 6))
test_performance_df.plot(kind='bar', ax=plt.gca())
plt.title('Model Performance Comparison - Test Set')
plt.xlabel('Metric')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.legend(title='Model')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Plot the validation set comparison
plt.figure(figsize=(12, 6))
val_performance_df.plot(kind='bar', ax=plt.gca())
plt.title('Model Performance Comparison - Validation Set')
plt.xlabel('Metric')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.legend(title='Model')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Combined plot for both test and validation
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

test_performance_df.plot(kind='bar', ax=axes[0])
axes[0].set_title('Test Set Performance')
axes[0].set_xlabel('Metric')
axes[0].set_ylabel('Score')
axes[0].set_ylim(0, 1)
axes[0].legend(title='Model')
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

val_performance_df.plot(kind='bar', ax=axes[1])
axes[1].set_title('Validation Set Performance')
axes[1].set_xlabel('Metric')
axes[1].set_ylabel('Score')
axes[1].set_ylim(0, 1)
axes[1].legend(title='Model')
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.suptitle('Model Performance Comparison', fontsize=16, y=1.05)
plt.show()

## 6. Example Predictions

In [ ]:
# Get some examples from the test set
examples = data['test']['texts'].iloc[:5].tolist()
true_labels = data['test']['labels'].iloc[:5].tolist()

# Get additional metadata for context
test_df = data['test']['df'].iloc[:5]
speakers = test_df['speaker'].tolist()
contexts = test_df['context'].tolist()
original_labels = test_df['label'].tolist()

# Make predictions with both models
logistic_preds = logistic_model.predict(examples)
distilbert_preds = distilbert_model.predict(examples)

# Map binary labels to text for better readability
label_text_map = {0: 'Real', 1: 'Fake'}
true_labels_text = [label_text_map[label] for label in true_labels]
logistic_preds_text = [label_text_map[pred] for pred in logistic_preds]
distilbert_preds_text = [label_text_map[pred] for pred in distilbert_preds]

# Display the results
results = pd.DataFrame({
    'Statement': [text[:100] + '...' for text in examples],
    'Speaker': speakers,
    'Context': contexts,
    'Original Label': original_labels,
    'Binary Label': true_labels_text,
    'Logistic Prediction': logistic_preds_text,
    'DistilBERT Prediction': distilbert_preds_text
})

results